In [1]:
# Importation des bibliothèques nécessaires
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from category_encoders import MEstimateEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Configuration par défaut de Matplotlib
plt.style.use("seaborn-v0_8-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

In [7]:
# Chargement du dataset Ames
df = pd.read_csv("ames.csv")
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YearSold,SaleType,SaleCondition,SalePrice
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141.0,31770.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,NaN,0.0,5,2010,WD,Normal,215000
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80.0,11622.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,0.0,No_Pool,Minimum_Privacy,NaN,0.0,6,2010,WD,Normal,105000
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81.0,14267.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,Gar2,12500.0,6,2010,WD,Normal,172000
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93.0,11160.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,NaN,0.0,4,2010,WD,Normal,244000
4,Two_Story_1946_and_Newer,Residential_Low_Density,74.0,13830.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,0.0,No_Pool,Minimum_Privacy,NaN,0.0,3,2010,WD,Normal,189900


---

# 1. Etude sur la variable cible « Prix de vente »
**Choix des caractéristiques à encoder**

Nous allos d’abord choisir les caractéristiques sur lesquelles appliquer un encodage cible. Les variables catégorielles avec un grand nombre de catégories sont souvent de bons candidats.

In [12]:
# Sélection des 3 variables catégorielles ayant le plus grand nombre de catégories
top_3_categorical = df.select_dtypes(["object"]).nunique().nlargest(3)
print(top_3_categorical)


Neighborhood    28
Exterior2nd     17
MSSubClass      16
dtype: int64


In [16]:
# sous la forme de liste
top_3_categorical_features = top_3_categorical.index.tolist()
print(top_3_categorical_features)


['Neighborhood', 'Exterior2nd', 'MSSubClass']


**Application de l’encodage cible**

On va appliquer l’encodage cible (**Target Encoding**) sur ces variables. Pour éviter le surapprentissage, on va d’abord diviser les données en un jeu d’encodage et un jeu d’entraînement. Ensuite, on utilisera la **M-Estimate Encoding** pour lisser les valeurs.

In [21]:
# Séparer une partie des données pour l'encodage
X_encode = df.sample(frac=0.20, random_state=0)
y_encode = X_encode.pop("SalePrice")

# Le reste pour l'entraînement
X_pretrain = df.drop(X_encode.index)
y_train = X_pretrain.pop("SalePrice")


#### Appliquer l’encodage cible avec M-Estimate Encoding
On va tester m = 5 et comparer les scores RMSLE.

In [26]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

def score_dataset(X, y, model=None):
    # Si le modèle est None, on crée un modèle par défaut
    if model is None:
        model = XGBRegressor()

    # Encodage des variables catégorielles
    X = X.copy()  # Évite de modifier le DataFrame original
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()

    # Vérification que le modèle est bien défini
    if model is None:
        raise ValueError("Le modèle passé à score_dataset est None. Assurez-vous de bien l'initialiser.")

    # La métrique utilisée pour cette compétition est le RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(model, X, y, cv=5, scoring="neg_mean_squared_log_error")

    score = -1 * score.mean()
    score = np.sqrt(score)

    return score


In [29]:
from category_encoders import MEstimateEncoder

# Création de l'encodeur avec les caractéristiques sélectionnées
m = 5
encoder = MEstimateEncoder(cols=top_3_categorical_features, m=m)

# Entraînement de l'encodeur sur le jeu d'encodage
encoder.fit(X_encode[top_3_categorical_features], y_encode)

# Transformation du jeu d'entraînement
X_train = X_pretrain.copy()
X_train[top_3_categorical_features] = encoder.transform(X_pretrain[top_3_categorical_features])


**Comparaison de score avec et sans encodage**

In [33]:
from sklearn.model_selection import cross_val_score

def score_dataset(X, y, model=None):
    if model is None:
        from sklearn.ensemble import RandomForestRegressor
        model = RandomForestRegressor(n_estimators=100)

    # Encodage des variables catégorielles si nécessaire
    X = X.copy()
    for colname in X.select_dtypes(["object"]):
        X[colname], _ = X[colname].factorize()

    # Évaluation du modèle avec cross-validation
    try:
        score = cross_val_score(
            model, X, y, cv=5, scoring="neg_mean_squared_log_error"
        )
        score = -1 * score.mean()
        return np.sqrt(score)
    except AttributeError as e:
        print(f"Erreur : {e}")
        return None


In [36]:
X = df.copy()
y = X.pop("SalePrice")

# Score sans encodage (baseline)
score_base = score_dataset(X, y)

# Score avec encodage
score_new = score_dataset(X_train, y_train)

print(f"Baseline Score: {score_base:.4f} RMSLE")
print(f"Score with Encoding: {score_new:.4f} RMSLE")

Baseline Score: 0.1436 RMSLE
Score with Encoding: 0.1438 RMSLE
